In [ ]:
# import packages
import pandas as pd
import json

In [343]:
file_paths = {
    'Bangalore': 'Raw_Datasets/bangalore_cars.xlsx',
    'Chennai': 'Raw_Datasets/chennai_cars.xlsx',
    'Delhi': 'Raw_Datasets/delhi_cars.xlsx',
    'Hyderabad': 'Raw_Datasets/hyderabad_cars.xlsx',
    'Jaipur': 'Raw_Datasets/jaipur_cars.xlsx',
    'Kolkata': 'Raw_Datasets/kolkata_cars.xlsx'
}

In [344]:
dataframes = []

def add_city_and_concatenate(file_paths):
    for city,path in file_paths.items():
        df = pd.read_excel(path)
        df["city"] = city
        dataframes.append(df)
    return pd.concat(dataframes,ignore_index=True)

concatenated_df = add_city_and_concatenate(file_paths)


In [345]:
def convert_jsonstr_to_dict(df):
    columns_to_be_checked = ['new_car_overview', 'new_car_feature', 'new_car_specs']
    for column in columns_to_be_checked:
        df[column] = df[column].apply(lambda x: eval(x) if isinstance(x,str) else x)
    return df

In [346]:
def format_new_car_overview(record):
    temp_dict = {}
    top_items = record.get("top",[])
    for item in top_items:
        key = item.get("key")
        value = item.get("value")
        temp_dict[key] = value
    return temp_dict

In [347]:
def format_new_car_feature(record):
    temp_list = []
    top_items = record.get("top", [])
    for item in top_items:
        temp_list.append(item['value'])

    data_items = record.get("data", [])
    for item in data_items:
        sub_items = item.get("list", [])
        for sub_item in sub_items:
            temp_list.append(sub_item['value'])

    return temp_list


In [348]:
def format_new_car_specs(record):
    temp_dict = {}

    top_items = record.get("top",[])
    for item in top_items:
        if 'key' in item and 'value' in item:
            temp_dict[item['key']] = item['value']

    data_items = record.get("data",[])
    for item in data_items:
        sub_items = item.get("list",[])
        for item in sub_items:
            if 'key' in item and 'value' in item:
                temp_dict[item['key']] = item['value']

    return temp_dict

In [349]:
def apply_data_extraction_to_columns(formatted_df):
    formatted_df['new_car_overview'] = formatted_df['new_car_overview'].apply(lambda x: format_new_car_overview(x) if isinstance(x,dict) else x)
    formatted_df['new_car_feature'] = formatted_df['new_car_feature'].apply(lambda x: format_new_car_feature(x) if isinstance(x,dict) else x)
    formatted_df['new_car_specs'] = formatted_df['new_car_specs'].apply(lambda x: format_new_car_specs(x) if isinstance(x,dict) else x)
    return formatted_df

In [350]:
concatenated_df = add_city_and_concatenate(file_paths)
formatted_df = convert_jsonstr_to_dict(concatenated_df)
df = apply_data_extraction_to_columns(formatted_df)


In [352]:
df.to_excel("df.xlsx",engine="openpyxl",index=False)